In [1]:
import ibmJupyterNotebookStyles
ibmJupyterNotebookStyles.apply_ibm_styles()

# $Z_2$ symmetry reduction and tapering

Tapering off qubits to simulate fermionic Hamiltonians: https://arxiv.org/abs/1701.08213

# Table of contents

- [Introduction](#Introduction)  
- [Z2 symmetry reduction](#Z2_Sym)  
- [Find symmetry group S](#Find_Symmetry)  
- [Find unitary Operator U](#Find_Unitary)  
- [Apply Z2 symmetry reduction to  $H_2$ molecule](#H2_Reduction)  
- [Demonstration with Qiskit](#Qiskit)  
- [Appendix](#Appendix)

<a id="Introduction"></a>
# Introduction

When we try to calculate the energy of the electric Hamiltonian using quantum computers, we can only use a limited number of qubits.  
A small number of qubits is preferred, to make the best use of current hardware.  
Some methods to reduce qubits from the Hamiltonian have been proposed.  
In this notebook, we will describe $Z_2$ symmetry reduction and demonstrate how many qubits will be reduced using Qiskit.   

We can summarize this problem:
1. What is the problem? Too many qubits are required.
1. Why is it a problem to use a large number of qubits? Qubits are "expensive" for NISQ hardware.
1. How do we solve the problem? We can reduce qubits by using $Z_2$ symmetry reduction.

The purposes of this notebook are:

- To explain $Z_2$ symmetry reduction method theoretically
- To apply this method to $H_2$ molecule with Qiskit.  

$H_2$ is one of the simplest molecules in chemistry and we target this molecule in this notebook.  
$H$ is the qubit Hamiltonian of $H_2$ molecule made by Jordan-Wigner mapping explained `03-01-2nd_Quantization_JW_Mapping`.  
$H$ has the following pauli operators listed in the left below image.  
Our goal is to explain how to transform $H$ to $H^{\prime}$ by Unitary Operator $U$ listed in the right below image.

<img src="figs/H2_compare.png" width="1000">

$H^{\prime}$ has three symmetries, and these symmetries generate corresponding operators $\sigma^x_2, \sigma^x_3$, and $\sigma^x_4$ respectively.  
These operators commutate with $H^{\prime}$:

$$
\begin{align}
[H^{\prime}, \sigma^x_i] = 0 \;\;\; (i=2,3,4)
\end{align}
$$

There exist simultaneous eigenstates of $H^{\prime}$ and $\sigma^x_i\;(i=2,3,4)$.  
$H^{\prime}$ includes these operators and we can  replace these operators with its eigenvalues $\pm 1$.  
This can lead to reducing three qubits from $H^{\prime}$.  
Only one qubit is required to calculate the energy of $H^{\prime}$.   

See [Appendix: Block Diagonalization](#block_diag)

<a id="Z2_Sym"></a>
# $Z_2$ Symmetry Reduction

We will next consider Hamiltonians with multiple $Z_2$ symmetries.  
$Z_2$ symmetries appear if a Hamiltonian has geometric symmetries such as rotations or reflections.

<details>
    <summary>Definition of $Z_2$ symmetry (Click to Expand)</summary>   
    
The $Z_n$ group describes a symmetry of a plane figure invariant after rotation of $2\pi/n$ degrees.  
$Z_2$ consists of two elements, $e$ and $\sigma$, such that $e \sigma=\sigma e = \sigma, \sigma^2=\sigma\sigma = e$.
</details>

$H$ is a qubit simulator Hamiltonian obtained by applying the Jordan-Wigner or Bravyi-Kitaev transformation to a second-quantized Hamiltonian:   

$$
\begin{align}
H = \sum^r_{j=1}h_j \boldsymbol{\sigma}_j
\end{align}
$$

Assume that $H$ consists of $M$ qubits and $r$ terms.  
Here, $h_j$ are some real coefficients and $\boldsymbol{\sigma}_j$ are $M$-qubit Pauli operators. That is, an $M$-fold tensor product of single-qubit Pauli operators $P_M$:   

$$
\begin{align}
P_M = \pm \{I, \sigma^x, \sigma^y, \sigma^z\}^{\otimes M}
\end{align}
$$

The purpose of this method is to find a set of operators $\{ \boldsymbol{\tau}_i \}^k_{i=1}$ that commute with all operators in $H$.

$$
\begin{align}
[\boldsymbol{\tau}_i, \boldsymbol{\sigma}_j] = 0 \;\;\;(\forall i,j)
\end{align}
$$

If we can find these operators, we can block-diagonalize $H$ and it might lead to a reduction of qubits.

We call this set of operators $\{ \boldsymbol{\tau}_i \}^k_{i=1}$  a symmetry group $S$. ($S \subseteq P_M$ and $-I \notin S$)  
$\boldsymbol{\tau}_i$ is also called a symmetry generator $S=\langle \boldsymbol{\tau}_1, \dots, \boldsymbol{\tau}_k \rangle$ and it can be constructed from: 

$$
\begin{align}
\boldsymbol{\tau}_i = U \sigma^x_i U^{\dagger}
\end{align}
$$

$U$ is a Clifford unitary operator such that $U \boldsymbol{\sigma} U^{\dagger} \in P_M$ for all $\boldsymbol{\sigma} \in P_M$.

Define a transformed Hamiltonian $H^{\prime}$:

$$
\begin{align}
H^{\prime} &= U^{\dagger} H U = \sum^r_{j=1}h_j\boldsymbol{\eta}_j\\
\boldsymbol{\eta}_j &= U^{\dagger} \boldsymbol{\sigma}_j U \in P_M
\end{align}
$$

Assuming that $[\boldsymbol{\tau}_i, \boldsymbol{\sigma}_j]=0$, we get the following commutation rules for $\sigma^x_i, \boldsymbol{\eta}_j$:

$$
\begin{align}
& U^{\dagger} \to [\boldsymbol{\tau}_i, \boldsymbol{\sigma}_j] \leftarrow U = 0 \\
& \Leftrightarrow [U^{\dagger} \boldsymbol{\tau}_i U, U^{\dagger} \boldsymbol{\sigma}_j U] = 0 \\
& \Leftrightarrow [\sigma^x_i, \boldsymbol{\eta}_j] = 0 \;\;\; (\forall i, j)
\end{align}
$$

This equation means that all terms in $H^{\prime}$ commute with all $X$-type Pauli operators on the first $k$ qubits, where $k$ is the total number of symmetries with commutative operators $\sigma^x_i, \boldsymbol{\tau}_i$.  
When searching for the ground energy of $H^{\prime}$  in the variational approach, we can replace these operators by their eigenvalues $\pm 1$.  

$$
\begin{align}
\{ \sigma^x_{i} \}^{k}_{i=1} \to \{\pm 1_i\}^{k}_{i=1}
\end{align}
$$

This means that we can remove the first $k$ qubits from $H^{\prime}$.

<details>
    <summary>Example (Click to expand)</summary>

The below image illustrates the case where $k=2\; (\sigma^x_1, \sigma^x_2)$.  
    
<img src="figs/block_diag_ex.png" width="500">

$H^{\prime}(\sigma^x_1, \sigma^x_2)$ is the Hamiltonian where we replaced $\sigma^x_1, \sigma^x_2$ with its eigenvalues($\pm 1$) respectively.  
This leads to the two qubits reduction from the original $H$.  
The total number of combinations is $2^2=4$. ($(\sigma^x_1, \sigma^x_2)=(1,1), (1,-1), (-1, 1), (-1, -1)$)  
Next we have to calculate all Hamiltonians and find the minimum energy among these Hamiltonians. 
</details>

Next, we will describe the following procedures in detail:  
1. Find Symmetry Group $S$: $S=\langle \boldsymbol{\tau}_1, \dots, \boldsymbol{\tau}_k \rangle$ such that $[\boldsymbol{\tau}_i, \boldsymbol{\sigma}_j]=0$ for all $\boldsymbol{\sigma}_j \in H$.
1. Find Unitary Operator $U$: $\boldsymbol{\tau}_i = U \sigma^x_i U^{\dagger}$

<a id="Find_Symmetry"></a>
# Find Symmetry Group $S$

## Notation of Pauli operators
First we will parametrize the Pauli operators $\boldsymbol{\sigma} \in P_M$ by using the binary string $(a_x|a_z)$ where $a_x, a_z \in \{ 0, 1 \}^M$ such that:  

$$
\begin{align}
\boldsymbol{\sigma}(a_x|a_z) = e^{i \phi} \prod^M_{i=1} (\sigma^x_i)^{a^i_x} \prod^M_{j=1} (\sigma^z_j)^{a^j_z}
\end{align}
$$

for some phase factor $e^{i \phi}$ that we will ignore later because it is a global phase factor.  
When $a^i_x=0$ or $a^i_z=0$, its operator becomes $I$ respectively.   
If $\boldsymbol{\sigma}$ contains $Y$-type Puali operators $\sigma^y_i$, add 1 to the $i$-th element of both $a_x$ and $a_z$.

<details>
    <summary>Examples ($M=4$) (Click to expand)</summary>
$$
\begin{align}
\sigma^x_1 \sigma^x_3 \sigma^z_2 \sigma^z_4 &\to a_x = (1, 0, 1, 0), a_z = (0, 1, 0, 1) \\
\sigma^y_1 \sigma^x_2 \sigma^z_3 &\to a_x = (1, 1, 0, 0), a_z = (1, 0, 1, 0) \\
\sigma^y_1 \sigma^x_1 \sigma^z_2 &\to a_x = (0, 0, 0, 0), a_z = (0, 1, 0, 0)
\end{align}
$$
</details>

## Commutation relations of $\boldsymbol{\sigma}(a_x|a_z)$
Consider two operators, $\boldsymbol{\sigma}(a_x|a_z) \boldsymbol{\sigma}(b_x|b_z)$, and change $\boldsymbol{\sigma}(a_x|a_z) \boldsymbol{\sigma}(b_x|b_z)$ to $\boldsymbol{\sigma}(b_x|b_z) \boldsymbol{\sigma}(a_x|a_z)$.  
Then, 

$$
\begin{align}
\boldsymbol{\sigma}(a_x|a_z) \boldsymbol{\sigma}(b_x|b_z) = (-1)^{a_x \cdot b_z + a_z \cdot b_x} \boldsymbol{\sigma}(b_x|b_z) \boldsymbol{\sigma}(a_x|a_z)
\end{align}
$$

This means that if $a_x \cdot b_z + a_z \cdot b_x = 0\;(mod\;2)$, $\boldsymbol{\sigma}(a_x|a_z)$ commutes with $\boldsymbol{\sigma}(b_x|b_z)$.

## Parity Check Matrix
The parity check matrix $E$ is used to find a symmetry group $\boldsymbol{\tau}_i$ of $H$.  
Each row of $E$ corresponds to $(a_z|a_x)$ of each term in $H$ as shown below:

<img src="figs/E.png" width="500">

$E$ has size $r \times (2M)$.  
Now we must find $\boldsymbol{\sigma}(b_z|b_x)$ such that $[\boldsymbol{\sigma}(a^i_z|a^i_x), \boldsymbol{\sigma}(b_z|b_x)]=0\;(\forall \boldsymbol{\sigma}(a^i_z|a^i_x) \in H_{sim})$.  
This can be done by solving the following equation:

$$
\begin{align}
E \cdot (b_x|b_z) = 0 \Leftrightarrow a^i_z \cdot b_x + a^i_x \cdot b_z = 0\;\;\; (\forall i \in r)
\end{align}
$$

This corresponds to finding $(b_x|b_z) \in \ker(E)$.  
We also have to find a maximal set of linearly independent vectors of $ker(E)$.  

<details>
    <summary>Detailed procedure to find  linearly independent $(b_z|b_x)$ (Click to expand)</summary>
Let $b_1 \cdots, b^d \in \{ 0,1 \}^{2M}$ be a set of linearly independent vectors that form a basis of $\ker(E)$.   
We shall write $b^i = (b^i_x|b^i_z)$. Let us choose the symmetry group $S$ as any maximal abelian subgroup of the group generated by the Pauli operators $\boldsymbol{\sigma}(b^1), \cdots, \boldsymbol{\sigma}(b^d)$.   
Such subgroup $S$ can be constructed, for example, by applying a symplectic version of the Gram-Schmidt orthogonalization to the set of basis vectors $b_1 \cdots, b^d \in \{ 0,1 \}^{2M}$, see [41]. It gives a maximal set of linearly independent vectors $g^1, \cdots, g^k \in span(b^1,\cdots,b^d)$ such that

$$
\begin{align}
g^i_x \cdot g^j_z + g^i_z \cdot g^j_x = 0 \;\;\;(mod\; 2)\;\;\;\forall 1\le i,j \le k
\end{align}
$$

Then generators of the symmetry group $S$ can be chosen as $\boldsymbol{\tau}_i = \boldsymbol{\sigma}(g^i_x|g^i_z)$ and $S = \langle \boldsymbol{\tau}_1, \cdots, \boldsymbol{\tau}_k \rangle$.
</details>  

These procedures can be summarized as follows:
1. Construct a parity check matrix $E$ from Pauli operators in $H$.
1. Find a maximal set of linearly independent vector of $ker(E)$.

<a id="Find_Unitary"></a>
# Find the unitary operator $U$

Let us consider how to find a unitary operator $U$.

$$
\begin{align}
\boldsymbol{\tau}_i = U \sigma^x_i U^{\dagger}\;\;\;(i=1,\cdots,k)
\end{align}
$$

We assume that the symmetry generators $\boldsymbol{\tau}_1, \cdots, \boldsymbol{\tau}_k$ are Z-type Pauli operators in all examples below.  
We impose the following condition to $U$:

$$
\begin{align}
\sigma^x_{q(i)} \boldsymbol{\tau}_j = (-1)^{\delta_{ij}} \boldsymbol{\tau}_j \sigma^x_{q(i)}
\end{align}
$$

$q(1), \cdots, q(k)$ are elements of a subset of qubits.  

Define unitary Clifford operators:

$$
\begin{align}
U_i = \frac{1}{\sqrt{2}}(\sigma^x_{q(i)} + \boldsymbol{\tau}_i)
\end{align}
$$

From its definition and the commutation rule, 

$$
\begin{align}
U^2_i &= I \\
U_i \sigma^x_{q(i)} U^{\dagger}_i &= \boldsymbol{\tau}_i \\
U_j \sigma^x_{q(i)} U^{\dagger}_j &= \sigma^x_i \;\;\; (i \neq j)
\end{align}
$$

<details>
    <summary>We can check these relations by straightforward calculations. (Click to Expand)</summary>
    
$$
\begin{align}
U^2_i &= \frac{1}{2}(\sigma^x_{q(i)} + \boldsymbol{\tau}_i)(\sigma^x_{q(i)} + \boldsymbol{\tau}_i) \\
&= \frac{1}{2}(2I + \sigma^x_{q(i)}\boldsymbol{\tau}_i + \boldsymbol{\tau}_i\sigma^x_{q(i)})\\
&= \frac{1}{2}(2I + \sigma^x_{q(i)}\boldsymbol{\tau}_i - \sigma^x_{q(i)}\boldsymbol{\tau}_i) = I
\end{align}
$$
    
$$
\begin{align}
U_i \sigma^x_{q(i)} U^{\dagger}_i &= \frac{1}{2}(\sigma^x_{q(i)} + \boldsymbol{\tau}_i) \sigma^x_{q(i)}(\sigma^x_{q(i)} + \boldsymbol{\tau}_i) \\ 
&=  \frac{1}{2}(I + \boldsymbol{\tau}_i\sigma^x_{q(i)})(\sigma^x_{q(i)} + \boldsymbol{\tau}_i) \\
&= \frac{1}{2}(\sigma^x_{q(i)} + 2\boldsymbol{\tau}_i + \boldsymbol{\tau}_i\sigma^x_{q(i)}\boldsymbol{\tau}_i) \\
&= \frac{1}{2}(\sigma^x_{q(i)} + 2\boldsymbol{\tau}_i - \sigma^x_{q(i)}\boldsymbol{\tau}_i\boldsymbol{\tau}_i) = \boldsymbol{\tau}_i
\end{align}
$$
    
$$
\begin{align}
U_j \sigma^x_{q(i)} U^{\dagger}_j &= \frac{1}{\sqrt{2}}(\sigma^x_{q(j)} + \boldsymbol{\tau}_j) \sigma^x_{q(i)}U^{\dagger}_j \\
&= \frac{1}{\sqrt{2}}(\sigma^x_{q(j)}\sigma^x_{q(i)} + \boldsymbol{\tau}_j\sigma^x_{q(i)})U^{\dagger}_j \\
&=  \sigma^x_{q(i)}\frac{1}{\sqrt{2}}(\sigma^x_{q(j)} + \boldsymbol{\tau}_j)U^{\dagger}_j = \sigma^x_{q(i)}
\end{align}
$$
</details>

And, $[U_i, U_j]=0$ because $\sigma^x_{q(i)}$ commutes with $\boldsymbol{\tau}_j$ for $i \neq j$.  
Thus a Clifford transformation $U$ can be constructed as $U=U_1U_2\cdots U_k$.  

$$
\begin{align}
\boldsymbol{\tau}_i &= U \sigma^x_{q(i)} U^{\dagger} \\
[H^{\prime}, \sigma^x_{q(i)}] &=0 \; (\forall \; i\in k)
\end{align}
$$

<a id="H2_Reduction"></a>
# Apply $Z_2$ symmetry reduction to an $H_2$ molecule

In this section, we apply this method to $H_2$ molecule.  
After Jordan-Wigner transformation, $H$ of $H_2$ molecule can be mapped to the following Pauli operators:

<img src="figs/H2_before.png" width="500">

$H$ consists of $M=4$ qubits and $r=14$ terms.  
First, we have to find symmetry of $H$ by calculateing $(b_z|b_x) \in ker(E)$.  

<details>
    <summary>The parity check matrix $E$ can be constructed from the above table (Click to expand)</summary>

- $r = 1$: $\sigma^z_1 \to (1000|0000)$
- $r = 2$: $\sigma^z_2 \to (0100|0000)$
- $r = 3$: $\sigma^z_3 \to (0010|0000)$
- $r = 4$: $\sigma^z_4 \to (0001|0000)$
- $r = 5$: $\sigma^z_1 \sigma^z_2 \to (1100|0000)$
- $r = 6$: $\sigma^z_1 \sigma^z_3 \to (1010|0000)$
- $r = 7$: $\sigma^z_1 \sigma^z_4 \to (1001|0000)$
- $r = 8$: $\sigma^z_2 \sigma^z_3 \to (0110|0000)$
- $r = 9$: $\sigma^z_2 \sigma^z_4 \to (0101|0000)$
- $r = 10$: $\sigma^z_3 \sigma^z_4 \to (0011|0000)$
- $r = 11$: $\sigma^y_1 \sigma^y_2 \sigma^x_3 \sigma^x_4 \to (1100|1111)$
- $r = 12$: $\sigma^x_1 \sigma^y_2 \sigma^t_3 \sigma^x_4 \to (0110|1111)$
- $r = 13$: $\sigma^y_1 \sigma^x_2 \sigma^x_3 \sigma^y_4\ \to (1001|1111)$
- $r = 14$: $\sigma^x_1 \sigma^x_2 \sigma^y_3 \sigma^y_4\ \to (0011|1111)$
</details>

Let $\tilde{E}$ be the row-echelon form of $E$ obtained by performing the Gaussian elimination.  
After removing zero row vectors, we get:

$$
\begin{align}
\tilde{E} = 
\left(
\begin{array}{cccc|cccc}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 1 & 1 & 1
\end{array}
\right)
\end{align}
$$

This result can be verified by executing the following code.

In [2]:
from sympy.matrices import Matrix

Make the parity check matrix $E$ from $(a_z|a_x)$ from Pauli operators in $H$.

In [3]:
E = Matrix([[1,0,0,0,0,0,0,0,0],
           [0,1,0,0,0,0,0,0,0],
           [0,0,1,0,0,0,0,0,0],
           [0,0,0,1,0,0,0,0,0],
           [1,1,0,0,0,0,0,0,0],
           [1,0,1,0,0,0,0,0,0],
           [1,0,0,1,0,0,0,0,0],
           [0,1,1,0,0,0,0,0,0],
           [0,1,0,1,0,0,0,0,0],
           [0,0,1,1,0,0,0,0,0],
           [1,1,0,0,1,1,1,1,0],
           [0,1,1,0,1,1,1,1,0],
           [1,0,0,1,1,1,1,1,0],
           [0,0,1,1,1,1,1,1,0]])

Get the row-echelon form of $E$ by `rref()`.

In [4]:
E.rref()[0]

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 1, 1, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0]])

This result is equal to $\tilde{E}$ for all zero rows and the furthest right columns.  

$$
\begin{align}
\tilde{E} = 
\left(
\begin{array}{cccc|cccc}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 1 & 1 & 1
\end{array}
\right)
\end{align}
$$

We can get three linearly independent vectors from $\tilde{E}$,

$$
\begin{align}
(0000|1100), \;\;\; (0000|1010), \;\;\; (0000|1001)
\end{align}
$$

These vectors give rise to three symmetry generators.

$$
\begin{align}
\boldsymbol{\tau}_1 &= \sigma^z_1\sigma^z_2 \\
\boldsymbol{\tau}_2 &= \sigma^z_1\sigma^z_3 \\
\boldsymbol{\tau}_3 &= \sigma^z_1\sigma^z_4
\end{align}
$$

Next we will define Clifford operators.  
Choose $\sigma^z_{q(i)} = \sigma^z_{i+1}$ for $i=1,2,3$, we can get each Clifford operator.

$$
\begin{align}
U_1 &= \frac{1}{\sqrt{2}}(\sigma^x_2 + \sigma^z_1 \sigma^z_2) \\
U_2 &= \frac{1}{\sqrt{2}}(\sigma^x_3 + \sigma^z_1 \sigma^z_3) \\
U_3 &= \frac{1}{\sqrt{2}}(\sigma^x_4 + \sigma^z_1 \sigma^z_4)
\end{align}
$$

Define $U=U_1U_2U_3$ and apply $UHU^{\dagger}$

<img src="figs/H2_compare.png" width="1000">

$$
\begin{align}
\boldsymbol{\tau}_1 &= U \sigma^x_2 U^{\dagger} \\
\boldsymbol{\tau}_2 &= U \sigma^x_3 U^{\dagger} \\
\boldsymbol{\tau}_3 &= U \sigma^x_4 U^{\dagger} \\
[H^{\prime}, \sigma^x_i] &=0 \; (i=2,3,4)
\end{align}
$$

$H^{\prime}$ includes $\sigma^x_2, \sigma^x_3, \sigma^x_4$.  
We can replace these terms with the eigenvalues $\pm 1$.  
As a result, only one qubit is required for $H^{\prime}$.   
We only have to consider $\sigma^x_1, \sigma^z_1$ terms as shown in the above table on the right.     
The original dimension of Hilbert space of $H^{\prime}$ is $2^4=16$.  
By using $Z_2$ symmetry reduction, we can decompose $H^{\prime}$ into $8(=2^3)$ Hamiltonians with dimension of $2$ ($=1$ qubit).    
We have to check which Hamiltonians have the minimum energy of $H^{\prime}$.  

Let's find the minimum energy by using the code in the following sections:

<a id="Qiskit"></a>
# Demonstration with Qiskit
In this section, we demonstrate how to reduce qubits from the original Hamiltonian of $H_2$ with Qiskit.  
First, the following libraries must first be imported:

In [5]:
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import TaperedQubitMapper

from qiskit_algorithms import VQE, NumPyMinimumEigensolver
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms.optimizers import COBYLA

from qiskit_aer.primitives import Estimator

from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Z2Symmetries

import qiskit_nature
qiskit_nature.settings.use_pauli_sum_op=False



Then we set up a PySCF driver.

In [6]:
molecule = MoleculeInfo(["H", "H"], [(0.0, 0.0, 0.0), (0.0, 0.0, 0.735)],
                     charge=0, multiplicity=1)
driver = PySCFDriver.from_molecule(molecule, basis="sto3g")

Next, we set up the electronic structure problem to generate the second quantized operator and a qubit converter that will map it to a qubit operator.

In [7]:
es_problem  = driver.run()
second_q_op = es_problem.hamiltonian.second_q_op()

print("\n".join(str(second_q_op).splitlines()[:10] + ["..."]))
# Second Quantized Operators

Fermionic Operator
number spin orbitals=4, number terms=36
  -1.25633907300325 * ( +_0 -_0 )
+ -0.47189600728114184 * ( +_1 -_1 )
+ -1.25633907300325 * ( +_2 -_2 )
+ -0.47189600728114184 * ( +_3 -_3 )
+ 0.3378550774017583 * ( +_0 +_0 -_0 -_0 )
+ 0.3322908651276482 * ( +_0 +_1 -_1 -_0 )
+ 0.3378550774017583 * ( +_0 +_2 -_2 -_0 )
+ 0.3322908651276482 * ( +_0 +_3 -_3 -_0 )
...



We can now transform this Hamiltonian to our qubit operator using a mapping scheme, such as Jordan-Wigner:

In [8]:
mapper=JordanWignerMapper()
qubit_op = mapper.map(second_q_op)
print(qubit_op)

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.81054798+0.j,  0.17218393+0.j, -0.22575349+0.j,  0.17218393+0.j,
 -0.22575349+0.j,  0.12091263+0.j,  0.16892754+0.j,  0.16614543+0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j,
  0.16614543+0.j,  0.17464343+0.j,  0.12091263+0.j])


These terms are slightly different from the following table:  
(For example, XXX, YYYY)

<img src="figs/H2_before.png" width="500">  

To learn more, refer to [Appendix: Check the operator difference in H2](#check_diff) at the end of this notebook.  
We will ignore this difference because the physics itself does not change.  

Before applying the $Z_2$ symmetry reduction, we will calculate the minimum energy of $H_2$.  

We prepare two optimization solvers such that `npme` and `vqe`.  
`npme` is the `NumPyMinimumEigensolver()` module and it uses exact-diagonalization.  
`vqe` is the `VQE` module from Qiskit and it uses Variational Quantum Eigensolver.

In [9]:
# ExactSolver
npme = NumPyMinimumEigensolver()

#  VQE
seed = 3
algorithm_globals.random_seed = seed
# estiamtor
estimator = Estimator(backend_options={'seed_simulator': seed})
# ansatz
ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')
# optimizer
iterations = 150
optimizer = COBYLA(maxiter=iterations)
vqe = VQE(estimator=estimator, ansatz=ansatz, optimizer=optimizer)

Next, we compute the energy of the $H_2$ Hamiltonian (`qubit_op`) before symmetry reduction by `npme` and `vqe` respectively.

In [10]:
result_npme = npme.compute_minimum_eigenvalue(operator=qubit_op)
result_vqe = vqe.compute_minimum_eigenvalue(operator=qubit_op)
min_energy_before_npme = result_npme.eigenvalue.real
min_energy_before_vqe = result_vqe.eigenvalue.real
print(f'Exact Energy: {min_energy_before_npme:.5f}')
print(f'VQE Energy: {min_energy_before_vqe:.5f}')

Exact Energy: -1.85728
VQE Energy: -1.83797


We now apply `z2symmetry_reduction` to `qubit_op` by setting eigenvalues to [1,-1,1].  
[1,-1,1] is just one of the eight possible patterns. 

In [11]:
z2_symmetries = Z2Symmetries.find_z2_symmetries(qubit_op)
z2_symmetries.tapering_values = [1,-1,1]
mapper_z2 = TaperedQubitMapper(mapper=mapper, z2symmetries=z2_symmetries)
qubit_op_z2 = mapper_z2.map(second_q_op)
print(qubit_op_z2)

SparsePauliOp(['I', 'Z'],
              coeffs=[-0.81626387+0.j,  0.34436787+0.j])


We can check symmetries (generators) and commutative operators.

In [12]:
z2_symmetries.settings

{'symmetries': [Pauli('ZIIZ'), Pauli('ZIZI'), Pauli('ZZII')],
 'sq_paulis': [Pauli('IIIX'), Pauli('IIXI'), Pauli('IXII')],
 'sq_list': [0, 1, 2],
 'tapering_values': [1, -1, 1]}

Compute the energy of the $H_2$ Hamiltonian (`qubit_op`) after applying symmetry reduction using the above pattern of eigenvalues.

In [13]:
result_npme = npme.compute_minimum_eigenvalue(operator=qubit_op_z2)
result_vqe = vqe.compute_minimum_eigenvalue(operator=qubit_op_z2)
min_energy_before_npme_z2 = result_npme.eigenvalue.real
min_energy_before_vqe_z2 = result_vqe.eigenvalue.real
print(f'Exact Energy: {min_energy_before_npme_z2:.5f}')
print(f'VQE Energy: {min_energy_before_vqe_z2:.5f}')

Exact Energy: -1.16063
VQE Energy: -1.16063


We have applied `z2symmetry_reduction` to only one of patterns of eigenvalues.  
We have to apply this reduction to all eight patterns and check which patterns have the minimum energy of $H_2$.

In [14]:
num_symmetries = 3
min_energy_after_vqe = 10**10
min_energy_after_npme = 10**10
min_z2symmetry_reduction = []
for bit in range(1<<num_symmetries):
    tapering_values = []
    for j in range(num_symmetries):
        if (bit & 1<<j):
            tapering_values.append(1)
        else:
            tapering_values.append(-1)
    print(f"{bit}: {tapering_values}")
    z2_symmetries.tapering_values = tapering_values
    mapper_z2 = TaperedQubitMapper(mapper=mapper, z2symmetries=z2_symmetries)
    qubit_op_z2 = mapper_z2.map(second_q_op)
    print(qubit_op_z2)
    print(z2_symmetries.settings)
    result_npme = npme.compute_minimum_eigenvalue(operator=qubit_op_z2)
    result_vqe = vqe.compute_minimum_eigenvalue(operator=qubit_op_z2)
    value_npme = result_npme.eigenvalue.real
    value_vqe = result_vqe.eigenvalue.real
    print(f'Exact value: {value_npme:.5f}')
    print(f'VQE value: {value_vqe:.5f} \n')
    if value_vqe < min_energy_after_vqe:
        min_energy_after_vqe = value_vqe
        min_energy_after_npme = value_npme
        min_z2symmetry_reduction = tapering_values

0: [-1, -1, -1]
SparsePauliOp(['I', 'Z'],
              coeffs=[-0.81626387+0.j, -0.34436787+0.j])
{'symmetries': [Pauli('ZIIZ'), Pauli('ZIZI'), Pauli('ZZII')], 'sq_paulis': [Pauli('IIIX'), Pauli('IIXI'), Pauli('IXII')], 'sq_list': [0, 1, 2], 'tapering_values': [-1, -1, -1]}
Exact value: -1.16063
VQE value: -1.16063 

1: [1, -1, -1]
SparsePauliOp(['I', 'X'],
              coeffs=[-1.06365335+0.j,  0.1809312 +0.j])
{'symmetries': [Pauli('ZIIZ'), Pauli('ZIZI'), Pauli('ZZII')], 'sq_paulis': [Pauli('IIIX'), Pauli('IIXI'), Pauli('IXII')], 'sq_list': [0, 1, 2], 'tapering_values': [1, -1, -1]}
Exact value: -1.24458
VQE value: -1.24458 

2: [-1, 1, -1]
SparsePauliOp(['I', 'Z', 'X'],
              coeffs=[-1.04109314+0.j, -0.79587485+0.j,  0.1809312 +0.j])
{'symmetries': [Pauli('ZIIZ'), Pauli('ZIZI'), Pauli('ZZII')], 'sq_paulis': [Pauli('IIIX'), Pauli('IIXI'), Pauli('IXII')], 'sq_list': [0, 1, 2], 'tapering_values': [-1, 1, -1]}
Exact value: -1.85728
VQE value: -1.86220 

3: [1, 1, -1]
SparsePa

After accumulating all the results, we can find the minimum energy among all patterns of eigenvalues.  
These are the results before and after applying Z2 symmetry reduction, as shown below:

In [15]:
print('Before Applying Z2 Symmetry Reduction:')
print(f'Exact Energy: {min_energy_before_npme:.5f}')
print(f'VQE Energy: {min_energy_before_vqe:.5f}\n')
print('After Applying Z2 Symmetry Reduction:')
print(f'Exact Energy: {min_energy_after_npme:.5f}')
print(f'VQE Energy: {min_energy_after_vqe:.5f}')
print('eigenvalues: {}'.format(min_z2symmetry_reduction))

Before Applying Z2 Symmetry Reduction:
Exact Energy: -1.85728
VQE Energy: -1.83797

After Applying Z2 Symmetry Reduction:
Exact Energy: -1.85728
VQE Energy: -1.86220
eigenvalues: [-1, 1, -1]


<a id="Appendix"></a>
# Appendix

<a id="block_diag"></a>
# Block Diagonalization
If a Hamiltonian $H$ commutes with a Hermitian operator $A$, there exists simultaneous eigenstates of both $H$ and $A$.  

$$
\begin{align}
[H, A] = 0 \to H|\psi\rangle = E|\psi\rangle \cap A|\psi\rangle=a|\psi\rangle
\end{align}
$$

Here, $E$ and $a$ are eigenvalues of $H$ and $A$, respectively.  
Because eigenstates of a Hermitian operator are orthogonal, we can block-diagonalize $H$ by using different eigenstates of $A$ as bases.  
  
Suppose that $A$ has the following two distinctive eigenstates and eigenvalues: 
$$
\begin{align}
A|\psi_1\rangle &= a_1|\psi_1\rangle \\
A|\psi_2\rangle &= a_2|\psi_1\rangle
\end{align}
$$

By using these states, we can construct the matrix elements of $H$ like this:

<img src="figs/block_diag.png" width="500">

If $H$ includes $A$ itself, $A$ can be replaced with a corresponding eigenvalue $a$.   

$$
\begin{align}
H &= \sigma^x_i + A + \cdots \to \sigma^x_i + a_1  + \cdots \;\;\;\text{for}\;|\psi_1\rangle\\
H &= \sigma^x_i + A + \cdots \to \sigma^x_i + a_2  + \cdots \;\;\;\text{for}\;|\psi_2\rangle
\end{align}
$$

For example, $A$ acts on only the first qubit. Other operators don't act on the first qubit.  
In this case, we can remove the first qubit from $H$ by replacing $A$ with its eigenvalues.  

<a id="check_diff"></a>
# Check the operator difference in $H_2$

The double-excitation operators transformed by Qiskit are slightly different from the following table in the paper.

In [16]:
mapper_z2 = TaperedQubitMapper(mapper=mapper, z2symmetries=z2_symmetries)
qubit_op_z2 = mapper_z2.map(second_q_op)
print(qubit_op_z2)

SparsePauliOp(['I', 'Z'],
              coeffs=[ 0.10713912+0.j, -0.10713912+0.j])


<img src="figs/H2_before.png" width="500">

This difference comes from the definition of operator indices of Qiskit. The double-excitation operators can be transformed as:<sup>1</sup>

<img src="figs/JWT_H2_double_excitation.png" width="600">

Observe that $h_1=-h_2, h_3=0$ in $H_2$.  
This indicates that the terms on red lines disappear.  
However, Qiskit defines the operator indices so that the terms on red line can remain and the other terms disapper.   
We will explore this difference below by extracting some functions from Qiskit source code.

In [17]:
import numpy as np
from qiskit.quantum_info.operators import Pauli, SparsePauliOp

In [18]:
def times_creation_op(position, pauli_table):
    # The creation operator is given by 0.5*(X + 1j*Y)
    real_part = SparsePauliOp(pauli_table[position][0], coeffs=[0.5])
    imag_part = SparsePauliOp(pauli_table[position][1], coeffs=[0.5j])
    return real_part + imag_part

def times_annihilation_op(position, pauli_table):
    # The annihilation operator is given by 0.5*(X - 1j*Y)
    real_part = SparsePauliOp(pauli_table[position][0], coeffs=[0.5])
    imag_part = SparsePauliOp(pauli_table[position][1], coeffs=[-0.5j])
    return real_part + imag_part

In [19]:
# nmodes = second_q_op[0].register_length
nmodes = second_q_op.register_length
pauli_table = []
for i in range(nmodes):
    a_z = np.asarray([1] * i + [0] + [0] * (nmodes - i - 1), dtype=bool)
    a_x = np.asarray([0] * i + [1] + [0] * (nmodes - i - 1), dtype=bool)
    b_z = np.asarray([1] * i + [1] + [0] * (nmodes - i - 1), dtype=bool)
    b_x = np.asarray([0] * i + [1] + [0] * (nmodes - i - 1), dtype=bool)
    pauli_table.append((Pauli((a_z, a_x)), Pauli((b_z, b_x))))

In [20]:
def JWTMAP(label_coeff_list):
    all_false = np.full(nmodes, False)
    ret_op_list = []
    for label, coeff in label_coeff_list:
        ret_op = SparsePauliOp(Pauli((all_false, all_false)), coeffs=[coeff])
        # Go through the label and replace the fermion operators by their qubit-equivalent, then
        # save the respective Pauli string in the pauli_str list.
        for position, char in enumerate(label):
            if char == "+":
                ret_op &= times_creation_op(position, pauli_table)
            elif char == "-":
                ret_op &= times_annihilation_op(position, pauli_table)
            elif char == "N":
                # The occupation number operator N is given by `+-`.
                ret_op &= times_creation_op(position, pauli_table)
                ret_op &= times_annihilation_op(position, pauli_table)
            elif char == "E":
                # The `emptiness number` operator E is given by `-+` = (I - N).
                ret_op &= times_annihilation_op(position, pauli_table)
                ret_op &= times_creation_op(position, pauli_table)
            elif char == "I":
                continue
        ret_op_list.append(ret_op)
    zero_op = SparsePauliOp(Pauli((all_false, all_false)), coeffs=[0])
    res_op = SparsePauliOp(sum(ret_op_list, zero_op)).simplify()
    return res_op

In [21]:
h = (0.18093119978423114+0j)
label_coeff_list_qiskit = [
    ('+-+-', h),
    ('+--+', h),
    ('-++-', h),
    ('-+-+', h)
]

label_coeff_list_original = [
    ('++--', -h),
    ('--++', -h),
    ('+--+', h),
    ('-++-', h)
]

In [22]:
op_qiskit = JWTMAP(label_coeff_list_qiskit)
print("Qiskit")
print(op_qiskit)

Qiskit
SparsePauliOp(['XYXY', 'YXXY', 'XYYX', 'YXYX'],
              coeffs=[-0.0452328+0.j,  0.0452328+0.j,  0.0452328+0.j, -0.0452328+0.j])


In [23]:
op_original = JWTMAP(label_coeff_list_original)
print("Original")
print(op_original)

Original
SparsePauliOp(['YXXY', 'XXYY', 'YYXX', 'XYYX'],
              coeffs=[ 0.0452328+0.j, -0.0452328+0.j, -0.0452328+0.j,  0.0452328+0.j])


## References

<a id='whitfield2010'></a>1. Whitfield, Biamonte, Aspuru-Guzik, [Simulation of Electronic Structure Hamiltonians Using Quantum Computers](https://arxiv.org/abs/1001.3855), *arXiv* 2010.<br>

---
*IBM - existing work*

<span style="font-size:10pt; font-weight:bold;"> &copy; Copyright IBM Corp. 2023 <br/>This content is categorized as an “existing” IBM asset as part of the IBM Quantum Accelerator contract. </span>
